In [175]:
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
import string
import json
import os
import re

In [176]:
stop_words = stopwords.words('english')

In [177]:
word_tokenize("w/")

['w/']

In [178]:
stop_words

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th

In [179]:
stemmer = SnowballStemmer("english")
stemmer.stem('POS_NUM')

u'pos_num'

In [180]:
# load NTUSD-Fin Model
with open("pretrained_model/NTUSD-Fin/NTUSD_Fin_word_v1.0.json", 'r') as f:
    fin_word = json.load(f)
with open("pretrained_model/NTUSD-Fin/NTUSD_Fin_hashtag_v1.0.json", 'r') as f:
    fin_hashtag = json.load(f)

fin_model = {}
for fin in fin_word:
    token = stemmer.stem(fin["token"])
    
    fin_model[token] = fin
for fin in fin_hashtag:
    fin_model['#'+token] = fin

In [181]:
market_sentiment = 0.0
for word in (fin_word + fin_hashtag):
    market_sentiment += word["market_sentiment"]

market_sentiment / len(fin_word + fin_hashtag)

0.5702130399167209

In [182]:
with open("data/training_set.json", 'r') as f:
    train_datas = json.load(f)

a = []
for data in train_datas:
    a.append(type(data["tweet"]))
print set(a)

set([<type 'unicode'>])


In [183]:
s = u"aaa"
type(s.encode('utf-8'))

str

In [282]:
snippet = "-13%  eee's  $coma"
snippet = re.sub(u"(?:[^a-zA-Z0-9\+\-$ ])", "", snippet)
snippet

'-13  eees  $coma'

In [298]:
def preprocess(datas):
    texts = []
    senti_scores = []
    for data in datas:
        
        snippet = data["snippet"]
        
        if type(snippet) == list:
            snippet = ' '.join(snippet)
        
        elif snippet == [] or snippet == "": # some data have empty snippet
            snippet = data["tweet"]
            
        raw_snippet = snippet

#         snippet = re.sub('https?://[A-Za-z0-9./]+', ' ', snippet)
#         snippet = re.sub(u"(?:[^a-zA-Z0-9\+\-$'#\/])", ' ', snippet)
#         print raw_snippet
#         print snippet
        
        # preprocess
        snippet = snippet.lower().replace(',', ' ').replace('.', ' ') \
                    .replace(u'（', ' ').replace(u'）', ' ').replace(u'(', ' ') \
                    .replace(u')', ' ').replace(u'%', ' ').replace(u'!', '').strip()
         
#         snippet = word_tokenize(snippet)
#         snippet = re.sub(u"(?:[^a-zA-Z0-9:，。%、！!？?,；～~.… ])", "", snippet)

        snippet = snippet.split(' ')
    
        # monitor
        if snippet == []:
            print raw_snippet

        # remove <num>, <comp> 
#         snippet = re.sub("\d+(?=\s)", "<num>", snippet)
#         snippet = re.sub(r'\s+\+<num>?(?=\s)', 'POS_NUM', snippet)
#         snippet = re.sub(r'\s+\-<num>?(?=\s)', 'NEG_NUM', snippet)
        
        snippet = [re.sub("\d+.*", "<num>", word) for word in snippet]         
        snippet = [re.sub(r'\+<num>.*', '_pos_num', word) for word in snippet]
        snippet = [re.sub(r'\-<num>.*', '_neg_num', word) for word in snippet]
        snippet = [word for word in snippet if "<num>" not in word]
        
        snippet = [word for word in snippet if word.startswith('http') == False]
        snippet = [word for word in snippet if word.startswith('$') == False]

        # word_tokenize
#         snippet = ' '.join(snippet).encode('utf-8')
#         snippet = snippet.translate(None, string.punctuation)
       
        snippet = ' '.join(snippet)
        snippet = word_tokenize(snippet)
        
        # stopwords includes some important words like "down"
#         snippet = [word for word in snippet if word not in stopwords.words('english')] # remove stopwords
        
        snippet = [stemmer.stem(word) for word in snippet] # stem words
        snippet = [word for word in snippet if word != '' ]
    
        new_snippet = snippet
#         print ' '.join(new_snippet)
        
#         print new_snippet
#         print raw_snippet
    
#         if new_snippet == []:
#             print "->" + raw_snippet
  
        # ignore empty training data
        if new_snippet != []:
            texts.append(new_snippet)
            senti_scores.append(data["sentiment"])
        
    return texts, senti_scores

In [299]:
len(train_datas)

1396

In [300]:
texts, senti_scores = preprocess(train_datas)

downgrade
downgrade
looking sexy this morning break on volume
looking sexy this morning break on volume
still long term fan!
still long term fan 
$TFM will have a way to go price wise to compete with Kroger. $KR
$TFM will have a way to go price wise to compete with Kroger  $KR
iPhone SE Could Be Doing Better Than Expected
iPhone SE Could Be Doing Better Than Expected
Now up to 200 stocks making new 52 week highs
Now up to 200 stocks making new 52 week highs
There's no reason why $GOOGL can't get back to $1,000
There's no reason why $GOOGL can't get back to $1 000
short
short
Pivotal sees 27% upside for Alphabet
Pivotal sees 27  upside for Alphabet
the most trending
the most trending
$GOOGL patents turn signal detector for autonomous cars.
$GOOGL patents turn signal detector for autonomous cars 
Investor long-term stock recommendation
Investor long-term stock recommendation
Breaking 52 week highs timing looks great now
Breaking 52 week highs timing looks great now
makes less profit than

$YHOO +2 61 
$TPLM Triangle #Petroleum 9.80+3% 2M Expanding Caliber
$TPLM Triangle #Petroleum 9 80+3  2M Expanding Caliber
Downbeaten stocks may be ripe for buys
Downbeaten stocks may be ripe for buys
ISRG Short
ISRG Short
$POZN +30
$POZN +30
-2.10% vs -0.90% Est Prior Revised from 25.40% to 17.60%
-2 10  vs -0 90  Est Prior Revised from 25 40  to 17 60 
looks constructive
looks constructive
just about to break out to new highs lots of technicals looking great right now
just about to break out to new highs lots of technicals looking great right now
Failed breakouts & failed setups Limping on crutches into next week
Failed breakouts   failed setups Limping on crutches into next week
shorting here into close might be play high beta reversing
shorting here into close might be play high beta reversing
good for economy
good for economy
gold going parabolic keep chasing those tops boys
gold going parabolic keep chasing those tops boys
Inside week buy triggers
Inside week buy triggers
revenue

5 Best Analyst Rated Stocks
5 Best Analyst Rated Stocks
Will still go down on lack of new products
Will still go down on lack of new products
I think $JAZZ has plenty of room to run and lots of momentum to continue pushing it to new highs
I think $JAZZ has plenty of room to run and lots of momentum to continue pushing it to new highs
dump it for minor loss
dump it for minor loss
Lets BURN!
Lets BURN 
every time it jumps back in range
every time it jumps back in range
Otherwise long
Otherwise long
A bottom right here?
A bottom right here 
Accumulating
Accumulating
Charter $CHTR Stock Gains Ahead of FCC Approval of Time Warner Cable Deal https://t.co/qbz1vC1IbT #StockAction
Charter $CHTR Stock Gains Ahead of FCC Approval of Time Warner Cable Deal   #StockAction
the most trending
the most trending
Anyone else see an issue with this. No wonder $ebay stock sucks
Anyone else see an issue with this  No wonder $ebay stock sucks
Unusual Call Activity
Unusual Call Activity
green, lol... make-up 

In [252]:
len(texts)

1394

In [253]:
def buildVocab(texts, word_count_threshold=2):
    # borrowed this function from NeuralTalk
    print 'preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, )

    word_counts = {}

    for text in texts:
        for w in text:
            word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

    print "after filter, vocab count: %d"% len(vocab)

    return vocab


In [254]:
def mapWordAndIdx(vocab):

	idx2word = {}
	idx2word[0] = '<pad>'
	idx2word[1] = '<bos>'
	idx2word[2] = '<eos>'
	idx2word[3] = '<unk>'

	word2idx = {}
	word2idx['<pad>'] = 0
	word2idx['<bos>'] = 1
	word2idx['<eos>'] = 2
	word2idx['<unk>'] = 3

	for idx, w in enumerate(vocab):
		word2idx[w] = idx + 4
		idx2word[idx+4] = w

	return word2idx, idx2word

In [255]:
vocab = buildVocab(texts)

preprocessing word counts and creating vocab based on word count threshold 2
after filter, vocab count: 804


In [256]:
word2idx, idx2word = mapWordAndIdx(vocab)

In [257]:
x_train = []
bos_idx = word2idx['<bos>']
eos_idx = word2idx['<eos>']
unk_idx = word2idx['<unk>']

for text in texts:
    #     x = [word2idx[word] if word in word2idx else unk_idx for word in text]
    x = [word2idx[word] for word in text if word in word2idx]
#     x = [bos_idx] + x + [eos_idx]
    x_train.append(x)

y_train = senti_scores

In [258]:
x_train = np.array(x_train)
y_train = np.array(y_train)

np.save("data/x_train", x_train)
np.save("data/y_train", y_train)

with open("data/word2idx.json", 'w') as f:
    json.dump(word2idx, f)

with open("data/idx2word.json", 'w') as f:
    json.dump(idx2word, f)

In [259]:
# test data

In [260]:
with open("data/test_set.json", 'r') as f:
    test_datas = json.load(f)

In [261]:
len(test_datas)

634

In [262]:
test_texts, test_senti_scores = preprocess(test_datas)

In [263]:
x_test = []
bos_idx = word2idx['<bos>']
eos_idx = word2idx['<eos>']
unk_idx = word2idx['<unk>']

for text in test_texts:
#     x = [word2idx[word] if word in word2idx else unk_idx for word in text]
    x = [word2idx[word] for word in text if word in word2idx]
#     x = [bos_idx] + x + [eos_idx]
    x_test.append(x)

y_test = test_senti_scores

In [264]:
len(x_test)

634

In [265]:
x_test = np.array(x_test)
y_test = np.array(y_test)

np.save("data/x_test", x_test)
np.save("data/y_test", y_test)

In [54]:
# check
score_sort = np.load('score_sort.npy')
pred = np.load('pred.npy')

sort_idx = score_sort.reshape([-1,]).argsort()
for idx in sort_idx[:20]:
    print "%f <--> %f" % (y_test[idx], pred[idx])
    print test_texts[idx]
    print test_datas[idx]
    print ""

-0.319000 <--> -0.318709
[u'alot', u'to', u'worri', u'about']
{u'snippet': u'alot to worry about', u'tweet': u'@MattKelmon 4 more years will be alot to worry about $spy $dia', u'target': u'$SPY', u'sentiment': -0.319}

-0.389000 <--> -0.389600
[u'anoth', u'sell', u'rate']
{u'snippet': u'Another Sell Rating', u'tweet': u'Isolationism? Another Sell Rating, Sell Ratings On SPY, QQQ, IWM And Several Companies https://t.co/6NKWEcrBA5 $AAPL $AXP $BAC $BLUE $FXE', u'target': u'$AXP', u'sentiment': -0.389}

-0.389000 <--> -0.389600
[u'anoth', u'sell', u'rate']
{u'snippet': u'Another Sell Rating', u'tweet': u'Isolationism? Another Sell Rating, Sell Ratings On SPY, QQQ, IWM And Several Companies https://t.co/6NKWEcrBA5 $AAPL $AXP $BAC $BLUE $FXE', u'target': u'$BAC', u'sentiment': -0.389}

-0.331000 <--> -0.329954
[u'bad', u'govern', u'not', u'confid', u'in', u'core', u'biz']
{u'snippet': [u'Bad governance', u'not confident in core biz'], u'tweet': u'@jay_21_ maybe $YHOO or $BABA. Bad governance

In [55]:

for idx in sort_idx[-20:]:
    print "%f <--> %f" % (y_test[idx], pred[idx])
    print test_texts[idx]
    print test_datas[idx]
    print ""

0.361000 <--> -0.281642
[u'sell', u'short', u'posit', u'on']
{u'snippet': u'Sell Short Position on', u'tweet': u'$CNP Sell Short Position on CNP,...Closed for Profit $ 59,367.00 (7.75%) http://stks.co/ghjc', u'target': u'$CNP', u'sentiment': 0.361}

0.543000 <--> -0.102388
[u'oil', u'back', u'abov']
{u'snippet': u'oil back above 60', u'tweet': u'$WRES moves better w/ oil anyways, oil back above 60', u'target': u'$WRES', u'sentiment': 0.543}

-0.193000 <--> 0.452947
[u'indic', u'are', u'ralli', u'today', u'a', u'lot', u'of', u'the', u'high', u'cover', u'compani', u"'", u'are', u'underperform']
{u'snippet': [u'Indices are rallying today', u"a lot of the 'highly covered companies' are underperforming"], u'tweet': u'Indices are rallying today, but it seems like a lot of the "highly covered companies" are underperforming $AAPL $FB $AMZN $MCD $SPY', u'target': u'$AMZN', u'sentiment': -0.193}

-0.329000 <--> 0.332659
[u'bullshit', u'!', u'all', u'#', u'applestor', u'are', u'empti', u'in', u'#